Data Mining Project:
Aaron, Sami, Wicky

How to get Rich? Or not?.....

Algorithms:
1. Categorical Prediction --> Naive Bayes
2. Clustering Algorithm   --> K-Means Clustering/Hierarchial Clustering
3. Continuous Values      --> Linear Regression/Random Forest

In [9]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import csv
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA

In [10]:
#imports dataset
data = pd.read_csv('adult.csv')
data.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [11]:
# Explicitly defined columns because the csv file didn't have column names
data.columns = ['Age', 'Workclass', 'Fnlwgt', 'Education', 'Education-Num', 'Maritial-Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Capital-Gain', 'Capital-Loss', 'Hours-Per-Week', 'Native-Country', 'Income']
data.head()

,Age,Workclass,Fnlwgt,Education,Education-Num,Maritial-Status,Occupation,Relationship,Race,Sex,Capital-Gain,Capital-Loss,Hours-Per-Week,Native-Country,Income
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [12]:
#Data PreProcessing Steps:
# Outliers
# Missing Data
# Duplicate????

#print("There are no Null/Missing Values")
data.isnull().sum()


Age                0
Workclass          0
Fnlwgt             0
Education          0
Education-Num      0
Maritial-Status    0
Occupation         0
Relationship       0
Race               0
Sex                0
Capital-Gain       0
Capital-Loss       0
Hours-Per-Week     0
Native-Country     0
Income             0
dtype: int64

In [17]:
#This changes the obj_data into numerical by using the labelEncoder (maybe leave in maybe take out?)
obj_data = data.select_dtypes(include=['object']).copy()
int_data = data.select_dtypes(include=['int64']).copy()
df = obj_data.apply(preprocessing.LabelEncoder().fit_transform)

df.head()
#int_data.head()


,Workclass,Education,Maritial-Status,Occupation,Relationship,Race,Sex,Native-Country,Income
0,6,9,2,4,0,4,1,39,0
1,4,11,0,6,1,4,1,39,0
2,4,1,2,6,0,2,1,39,0
3,4,9,2,10,5,2,0,5,0
4,4,12,2,4,5,4,0,39,0


In [18]:
# Need to find and remove outliers from the dataset
#this doesn't work atm -->
df.describe()

,Workclass,Education,Maritial-Status,Occupation,Relationship,Race,Sex,Native-Country,Income
count,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000,32560.000000
mean,3.868796,10.298249,2.611794,6.572912,1.446376,3.665848,0.669195,36.718796,0.240817
std,1.455879,3.870317,1.506225,4.228809,1.606794,0.848817,0.470510,7.823892,0.427586
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,9.000000,2.000000,3.000000,0.000000,4.000000,0.000000,39.000000,0.000000
50%,4.000000,11.000000,2.000000,7.000000,1.000000,4.000000,1.000000,39.000000,0.000000
75%,4.000000,12.000000,4.000000,10.000000,3.000000,4.000000,1.000000,39.000000,0.000000
max,8.000000,15.000000,6.000000,14.000000,5.000000,4.000000,1.000000,41.000000,1.000000


In [19]:
#finding the interquartile range (IQR)
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

Workclass          0.0
Education          3.0
Maritial-Status    2.0
Occupation         7.0
Relationship       3.0
Race               0.0
Sex                1.0
Native-Country     0.0
Income             0.0
dtype: float64


In [25]:
#the values that are 'True' represent the presence of the outlier
print(df < (Q1 - 1.5 * IQR)) or (df > (Q3 + 1.5 * IQR))

       Workclass  Education  Maritial-Status  Occupation  Relationship   Race  \
0          False      False            False       False         False  False   
1          False      False            False       False         False  False   
2          False       True            False       False         False   True   
3          False      False            False       False         False   True   
4          False      False            False       False         False  False   
...          ...        ...              ...         ...           ...    ...   
32555      False      False            False       False         False  False   
32556      False      False            False       False         False  False   
32557      False      False            False       False         False  False   
32558      False      False            False       False         False  False   
32559      False      False            False       False         False  False   

         Sex  Native-Countr

,Workclass,Education,Maritial-Status,Occupation,Relationship,Race,Sex,Native-Country,Income
0,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
32555,False,False,False,False,False,False,False,False,False
32556,False,False,False,False,False,False,False,False,True
32557,False,False,False,False,False,False,False,False,False
32558,False,False,False,False,False,False,False,False,False
